BEFORE EXECUTING, need to create the database of all genomes

In [1]:
import os
import Bio
from Bio import SeqIO
from Bio.Blast.Applications import NcbiblastnCommandline
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC
from Bio.SeqRecord import SeqRecord
import xml.etree.ElementTree as ET
from openpyxl import Workbook
from openpyxl.styles import Color, PatternFill, Style

In [58]:
q = './STDB/tempfile.fasta'
d = './BLAST_DB/DavisDB.fasta'

for item in SeqIO.parse('./STDB/ProtDB.fasta','fasta'):
    tempfile = './STDB/tempfile.fasta'
    otf = open(tempfile,'w')
    SeqIO.write(item, tempfile, 'fasta')
    otf.close
    o = './STX_BLAST_HITS/'+item.id+'.xml'
    os.system(str(NcbitblastnCommandline(query = q, db = d, out = o, outfmt=5)))
os.remove(tempfile)            

In [56]:
q = './STDB/nuctest.fasta'
d = './BLAST_DB/DavisDB.fasta'
o = './STX_BLAST_HITS/wtf.xml'

os.system(str(NcbiblastnCommandline(query = q, db = d, out = o, outfmt=5)))

0

In [2]:
#Extract BLAST hits

hitdict = {}

for file in os.listdir('./STX_BLAST_HITS'):
    if file.endswith('.xml'):
        queryg = os.path.splitext(file)[0]
        queryf = os.path.join('./STX_BLAST_HITS/'+file)
        tree = ET.parse(queryf)
        root = tree.getroot()
            
        for iteration in root.findall(".//Iteration"):
            qlen = iteration.find('Iteration_query-len').text
        for hit in root.findall(".//Hit"):
            contig = hit.find('Hit_id').text
            for hsp in hit.findall('.//Hit_hsps/Hsp'):
                hseq = hsp.find('Hsp_hseq').text
                qstart = hsp.find('Hsp_query-from').text
                qstop = hsp.find('Hsp_query-to').text
                hstart = hsp.find('Hsp_hit-from').text
                hstop = hsp.find('Hsp_hit-to').text
                alen = hsp.find('Hsp_align-len').text
                ids = hsp.find('Hsp_identity').text
                pos = hsp.find('Hsp_positive').text
                gaps = hsp.find('Hsp_gaps').text
                
            hitdict[contig+'|'+queryg]=[qstart,qstop,qlen,alen,ids,pos,gaps]

In [179]:
#Create lists of all genomes and genes - used to index information by item name!
wb = Workbook()
ws1 = wb.create_sheet(0)
ws2 = wb.create_sheet(1)
ws3 = wb.create_sheet(2)
ws4 = wb.create_sheet(3)
ws1.title = 'gene locations'
ws2.title = 'hit count'
ws3.title = '%ID, hit len, hit range'
ws4.title = '%ID, %POS, Gaps%'
gn = ['0','1',]
hn = ['0','1',]
keys = hitdict.keys()

#Flipped.
for item in keys:
    hn.append(item.split('|')[0].split('(')[0])
    gn.append(item.split('|')[1])
#Set function removes duplicates, list makes it a list, sorted sorts the list
gn = sorted(list(set(gn)))
hn = sorted(list(set(hn)))

#Labels, labels in the first row and column are important...
for n in range(2, len(gn)):
    ws1.cell(row = n, column = 1).value = gn[n]
    ws2.cell(row = n, column = 1).value = gn[n]
    ws3.cell(row = n, column = 1).value = gn[n]
    ws4.cell(row = n, column = 1).value = gn[n]
for n in range(2, len(hn)):
    ws1.cell(row = 1, column = n).value = hn[n]
    ws2.cell(row = 1, column = n).value = hn[n]
    ws3.cell(row = 1, column = n).value = hn[n]
    ws4.cell(row = 1, column = n).value = hn[n]
    
#makes the file
#example dict: S87(NODE_65)|EH250_2b_A

for item in hitdict:
    genome = item.split('(')[0]
    contig = item.split('(')[1].split(')')[0]
    gene = item.split('|')[1]
    ids = hitdict[item][4]
    pos = hitdict[item][5]
    gaps = hitdict[item][6]
    qlen = hitdict[item][2]
    alen = hitdict[item][3]
    qstart=hitdict[item][0]
    qstop=hitdict[item][1]
    hi = hn.index(genome)
    gi = gn.index(gene)
    old = ws3.cell(row = gi, column = hi).value
    if old is None:
        old = ''
    ws3.cell(row = gi, column = hi).value = old+' || '+'ID: '+item.split('|')[0]+', {0:.2f}'.format(int(ids)*100/int(alen))+'%ID, len: '+alen+', range: '+qstart+'-'+qstop
    old = ws4.cell(row = gi, column = hi).value
    if old is None:
        old = ''
    ws4.cell(row = gi, column = hi).value = old+' || '+'{0:.2f}'.format(int(ids)/int(alen))+', '+'{0:.2f}'.format(int(pos)/int(qlen))+', '+'{0:.2f}'.format(int(gaps)/int(qlen))
wb.save('gene_locations.xlsx')

In [ ]:
#
for gi in range(2, len(gn)):
    for hi in range(2,len(hn)):
        if ws2.cell(row = (gi), column = (hi)).value is not None:
            bounds = []
            keys = hitdict.keys()
            keylist = [item for item in keys]
            keyhandles = [item.split('(')[0] for item in keylist]
                    
                    newcellvalue = str(ws2.cell(row = gi, column =  hi).value) + '('+pctcover+'% coverage)'
                    ws2.cell(row = gi, column = hi).value = newcellvalue
wb.save('gene_locations.xlsx')

In [27]:
m = hitdict.keys()
k = [item.split('(')[0] for item in m]
print(k)
for i in gn:
    print(i, k.count(i))

['S87', 'S20', 'S04', 'S08', 'S71', 'S32', 'S42', 'S27', 'S83', 'S41', 'S61', 'S13', 'S67', 'S32', 'S16', 'S24', 'S85', 'S06', 'S11', 'S82', 'S38', 'S56', 'S27', 'S10', 'S36', 'S19', 'S74', 'S33', 'S37', 'S78', 'S36', 'S42', 'S58', 'S30', 'S88', 'S44', 'S73', 'S18', 'S77', 'S60', 'S74', 'S58', 'S20', 'S63', 'S04', 'S85', 'S75', 'S31', 'S67', 'S08', 'S61', 'S86', 'S34', 'S07', 'S77', 'S80', 'S36', 'S41', 'S49', 'S83', 'S68', 'S11', 'S06', 'S17', 'S04', 'S38', 'S19', 'S23', 'S28', 'S36', 'S84', 'S17', 'S86', 'S57', 'S11', 'S16', 'S05', 'S10', 'S57', 'S81', 'S77', 'S42', 'S31', 'S78', 'S34', 'S20', 'S67', 'S44', 'S71', 'S72', 'S45', 'S83', 'S02', 'S07', 'S67', 'S25', 'S16', 'S25', 'S78', 'S06', 'S36', 'S23', 'S42', 'S82', 'S81', 'S35', 'S68', 'S57', 'S67', 'S40', 'S17', 'S14', 'S07', 'S74', 'S31', 'S10', 'S71', 'S11', 'S39', 'S51', 'S75', 'S65', 'S31', 'S23', 'S71', 'S19', 'S83', 'S29', 'S16', 'S19', 'S78', 'S56', 'S40', 'S77', 'S39', 'S71', 'S32', 'S30', 'S76', 'S88', 'S58', 'S07', 'S76'

In [176]:
keylib = [item for item in hitdict.keys()]
contiglist = [item.split('|')[0] for item in keylib]
contiglist = sorted(set(contiglist))
for item in contiglist:
    #Just the keys
    keylist = []
    #Raw scores pulled from hitdict (align len/query len, IDs/aln len)
    Alenlist = []
    Aidlist = []
    Blenlist = []
    Bidlist = []
    Amaxlenlist = []
    Amaxidlist = []
    Akeylist = []
    Bmaxlenlist = []
    Bmaxidlist = []
    Bkeylist = []
    for key in keylib:
        if item in key:
            keylist.append(key)
    for key in keylist:
        if key.split('_')[-1]=='A':
            Alenlist.append(int(hitdict[key][3])/int(hitdict[key][2]))
            Aidlist.append(int(hitdict[key][4])/int(hitdict[key][3]))
            Blenlist.append(0)
            Bidlist.append(0)
        if key.split('_')[-1]=='B':
            Blenlist.append(int(hitdict[key][3])/int(hitdict[key][2]))
            Bidlist.append(int(hitdict[key][4])/int(hitdict[key][3]))
            Alenlist.append(0)
            Aidlist.append(0)
    if Alenlist is not []:
        Amaxlen = max(Alenlist)
    if Aidlist is not []:
        Amaxid = max(Aidlist)
    if Blenlist is not []:
        Bmaxlen = max(Blenlist)
    if Bidlist is not []:
        Bmaxid = max(Bidlist)
    
    try:
        Amaxlenlist = [i for i in keylist if (Alenlist[keylist.index(i)] == Amaxlen)]
    except:
        pass
    try:
        Amaxidlist = [i for i in keylist if (Aidlist[keylist.index(i)] == Amaxid)]
    except:
        pass    
    try:
        Bmaxlenlist = [i for i in keylist if (Blenlist[keylist.index(i)] == Bmaxlen)]
    except:
        pass
    try:
        Bmaxidlist = [i for i in keylist if (Bidlist[keylist.index(i)] == Bmaxid)]
    except:
        pass
    print(item)
    if Amaxlen is 0:
        print('No hit')
    else:
        print(Amaxlenlist)
    
    if Amaxid is 0:
        print('No hit')
    else:
        print(Amaxidlist)
    
    if Bmaxlen is 0:
        print('No hit')
    else:
        print(Bmaxlenlist)
    
    if Bmaxid is 0:
        print('No hit')
    else:
        print(Bmaxidlist)
    print('\n')

S01(NODE_1)
['S01(NODE_1)|Sakai_2a_A', 'S01(NODE_1)|94C_2a_A']
['S01(NODE_1)|Sakai_2a_A', 'S01(NODE_1)|94C_2a_A']
No hit
No hit


S01(NODE_117)
['S01(NODE_117)|Sakai_2a_A', 'S01(NODE_117)|94C_2a_A', 'S01(NODE_117)|EH250_2b_A', 'S01(NODE_117)|AJ966783_2g_A', 'S01(NODE_117)|AJ313016_2e_A', 'S01(NODE_117)|EC5.11_2c_A', 'S01(NODE_117)|AF298816_2d_A']
['S01(NODE_117)|EC5.11_2c_A']
['S01(NODE_117)|EH250_2b_B', 'S01(NODE_117)|AJ313016_2e_B', 'S01(NODE_117)|AJ966783_2g_B', 'S01(NODE_117)|AJ270998_2f_B', 'S01(NODE_117)|EC5.11_2c_B', 'S01(NODE_117)|AF298816_2d_B', 'S01(NODE_117)|Sakai_2a_B', 'S01(NODE_117)|94C_2a_B']
['S01(NODE_117)|EC5.11_2c_B']


S01(NODE_127)
['S01(NODE_127)|94C_2a_A', 'S01(NODE_127)|EC5.11_2c_A', 'S01(NODE_127)|AF298816_2d_A', 'S01(NODE_127)|AJ966783_2g_A', 'S01(NODE_127)|Sakai_2a_A', 'S01(NODE_127)|EH250_2b_A', 'S01(NODE_127)|AJ313016_2e_A']
['S01(NODE_127)|94C_2a_A', 'S01(NODE_127)|Sakai_2a_A']
['S01(NODE_127)|Sakai_2a_B', 'S01(NODE_127)|AF298816_2d_B', 'S01(NODE_127)|AJ31

In [144]:
print(keylist)

['S88(NODE_73)|94C_2a_B', 'S88(NODE_73)|AJ270998_2f_A', 'S88(NODE_73)|AJ966783_2g_A', 'S88(NODE_73)|EC5.11_2c_A', 'S88(NODE_73)|Sakai_2a_A', 'S88(NODE_73)|Sakai_1_A', 'S88(NODE_73)|AF298816_2d_B', 'S88(NODE_73)|Sakai_2a_B', 'S88(NODE_73)|EC5.11_2c_B', 'S88(NODE_73)|EH250_2b_A', 'S88(NODE_73)|AJ313016_2e_B', 'S88(NODE_73)|Sakai_1_B', 'S88(NODE_73)|AJ270998_2f_B', 'S88(NODE_73)|EH250_2b_B', 'S88(NODE_73)|AF298816_2d_A', 'S88(NODE_73)|AJ313016_2e_A', 'S88(NODE_73)|AJ966783_2g_B', 'S88(NODE_73)|94C_2a_A']


In [145]:
H = [i.split('_')[-1] for i in keylist]
print(H)

['B', 'A', 'A', 'A', 'A', 'A', 'B', 'B', 'B', 'A', 'B', 'B', 'B', 'B', 'A', 'A', 'B', 'A']


In [143]:
print(Bidlist)

[0.5697674418604651, '0', '0', '0', '0', '0', 0.6142857142857143, 0.5697674418604651, 0.5581395348837209, '0', 0.5581395348837209, 1.0, 0.5813953488372093, 0.6046511627906976, '0', '0', 0.5813953488372093, '0']


In [152]:
print(Alenlist)

[0, 0.9090909090909091, 0.9843260188087775, 0.9780564263322884, 0.9780564263322884, 1.0, 0, 0, 0, 0.9278996865203761, 0, 0, 0, 0, 0.9278996865203761, 0.9310344827586207, 0, 0.9780564263322884]


In [153]:
print(Amaxidlist)

[]


In [156]:
if Alenlist is not []:
    print((Alenlist))

[0, 0.9090909090909091, 0.9843260188087775, 0.9780564263322884, 0.9780564263322884, 1.0, 0, 0, 0, 0.9278996865203761, 0, 0, 0, 0, 0.9278996865203761, 0.9310344827586207, 0, 0.9780564263322884]
